In [56]:
import pandas as pd
import numpy as np
import os

''' #i
df = pd.DataFrame()
dirs = os.listdir(".\ETFs")
print(dirs)
i = 0
for d in dirs:
    files = os.listdir(f".\ETFs\{d}")
    for f in files:
        a = pd.read_csv(f".\ETFs\{d}\{f}")
        a = a[a.columns[:-1]]
        a["Sector"]=d
        a["ETF"]=f.split(".")[0].upper()
        a = a[["Date","ETF","Sector"]+list(a.columns[1:-2])]
        #print(a)
        df = pd.concat([df,a],ignore_index=True)

df.to_csv("data.csv") # so that the computation wouldn't take this long next time
'''

ETFdf = pd.read_csv("data.csv")
ETFdf = ETFdf[ETFdf.columns[1:]]
ETFdf = ETFdf[(ETFdf["Date"]>="2008-01-25") & (ETFdf["Date"]<="2016-11-22")] # the data for the other dataset is between these dates
ETFdf
#ETFdf[ETFdf["Date"]=="YYYY-MM-DD"]# return stock data on a specific date

Rdtdf = pd.read_csv("reddit_data.csv")
Rdtdf = Rdtdf[Rdtdf.columns[:-3]]
Rdtdf


,time_created,date_created,up_votes,down_votes,title
0,1201232046,2008-01-25,3,0,Scores killed in Pakistan clashes
1,1201232075,2008-01-25,2,0,Japan resumes refuelling mission
2,1201232523,2008-01-25,3,0,US presses Egypt on Gaza border
3,1201233290,2008-01-25,1,0,Jump-start economy: Give health care to all
4,1201274720,2008-01-25,4,0,Council of Europe bashes EU&UN terror blacklist
...,...,...,...,...,...
509231,1479816764,2016-11-22,5,0,Heil Trump : Donald Trump s alt-right white...
509232,1479816772,2016-11-22,1,0,There are people speculating that this could b...
509233,1479817056,2016-11-22,1,0,Professor receives Arab Researchers Award
509234,1479817157,2016-11-22,1,0,Nigel Farage attacks response to Trump ambassa...
